In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import joblib
from crewai import Agent, Task, Crew, Process

# Configuration
CONFIDENCE_THRESHOLD = 0.6
MODEL_SAVE_PATH = "email_classifier.joblib"
VECTORIZER_SAVE_PATH = "tfidf_vectorizer.joblib"

class EmailClassifierAgent:
    def __init__(self):
        self.model = None
        self.vectorizer = None
    
    def load_model(self):
        """Load pre-trained model"""
        try:
            self.model = joblib.load(MODEL_SAVE_PATH)
            self.vectorizer = joblib.load(VECTORIZER_SAVE_PATH)
            print("Model loaded successfully\n")
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            exit()

    def classify_email(self, email_text):
        """Classify an email"""
        try:
            text = email_text.lower().strip()
            
            if not self.model or not self.vectorizer:
                raise ValueError("Model not loaded")
            
            # Vectorize and predict
            vec = self.vectorizer.transform([text])
            prediction = self.model.predict(vec)[0]
            proba = self.model.predict_proba(vec)[0]
            confidence = np.max(proba)
            
            return {
                "email_text": email_text,
                "predicted_category": prediction,
                "confidence": float(confidence),
                "probabilities": {cls: float(prob) for cls, prob in zip(self.model.classes_, proba)}
            }
        except Exception as e:
            return {"error": str(e)}

class ResponseGeneratorAgent:
    def generate_response(self, classification_result):
        """Generate a response based on the classification"""
        category = classification_result['predicted_category']
        
        if category == "IT":
            return """Dear Sender,

Thank you for contacting IT support. We've received your request regarding your technical issue. 
Could you please provide more details about the problem you're experiencing?

Best regards,
IT Support Team"""
        elif category == "HR":
            return """Dear Sender,

Thank you for reaching out to HR. We've received your message and will respond 
to your inquiry within 24 hours. If this is urgent, please call our HR hotline.

Best regards,
Human Resources"""
        else:
            return """Dear Sender,

Thank you for your email. We've received your message and will respond 
as soon as possible. For urgent matters, please contact our support line.

Best regards,
Customer Service"""

class EscalationManagerAgent:
    def __init__(self):
        self.threshold = CONFIDENCE_THRESHOLD
    
    def handle_result(self, classification_result):
        """Decide whether to proceed or escalate"""
        if classification_result['confidence'] < self.threshold:
            return {
                "action": "escalate",
                "reason": f"Low confidence ({classification_result['confidence']*100:.1f}%)",
                "original_result": classification_result
            }
        else:
            return {
                "action": "proceed",
                "original_result": classification_result
            }

def main():
    # Initialize agents
    classifier = EmailClassifierAgent()
    classifier.load_model()
    
    response_agent = ResponseGeneratorAgent()
    escalation_agent = EscalationManagerAgent()
    
    # Interactive loop
    while True:
        print("\n" + "="*50)
        print("Company Email Classification System")
        print("="*50)
        print("\nPlease enter the email text (or type 'exit' to quit):")
        email_text = input("> ")
        
        if email_text.lower() == 'exit':
            print("\nExiting the system. Goodbye!")
            break
            
        if not email_text.strip():
            print("\nPlease enter some text for classification")
            continue
            
        # Classify the email
        print("\nClassifying email...")
        classification_result = classifier.classify_email(email_text)
        
        if 'error' in classification_result:
            print(f"\nError: {classification_result['error']}")
            continue
            
        # Display classification results
        print("\nClassification Results:")
        print(f"Predicted Category: {classification_result['predicted_category']}")
        print(f"Confidence: {classification_result['confidence']*100:.1f}%")
        
        # Generate response
        response = response_agent.generate_response(classification_result)
        
        # Check if we should escalate
        decision = escalation_agent.handle_result(classification_result)
        
        if decision['action'] == 'escalate':
            print(f"\n[SYSTEM] This email requires human review: {decision['reason']}")
            print("\nSuggested Response (for human review):")
        else:
            print("\nGenerated Response:")
            
        print(response)
        
        print("\n" + "-"*50)
        input("Press Enter to classify another email...")

if __name__ == "__main__":
    main()

Model loaded successfully


Company Email Classification System

Please enter the email text (or type 'exit' to quit):


>  how are you



Classifying email...

Classification Results:
Predicted Category: Other
Confidence: 48.0%

[SYSTEM] This email requires human review: Low confidence (48.0%)

Suggested Response (for human review):
Dear Sender,

Thank you for your email. We've received your message and will respond 
as soon as possible. For urgent matters, please contact our support line.

Best regards,
Customer Service

--------------------------------------------------
